In [ ]:
import json
import pandas as pd
import numpy as np
import a1chemy as a1
import a1chemy.indicators as indicators
import a1chemy.data_source as data_source
from tqdm.notebook import tqdm
import random
import time
import pymongo
from IPython.core.display import display, HTML
from a1chemy.common.tag import Tree
from a1chemy.common import Tag
import itertools

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client=mongo_client)



root = mongo_tags_client.find(id='swsindex')
tree = Tree(root=root)
parent_id_list = [root.id]
while parent_id_list:
    print(parent_id_list)
    children_dict = mongo_tags_client.find_children(parent_list=parent_id_list)
    print('children_dict len={}'.format(len(children_dict)))
    next_parent_id_list = []
    for parent, children in children_dict.items():
        print('parent:{} len:{}'.format(parent, len(children)))
        tree.add_relation(parent=parent, children=children)
        current_children_id_list = [child.id for child in children]
        next_parent_id_list.extend(current_children_id_list)
    parent_id_list = next_parent_id_list

In [ ]:
children_dict = mongo_tags_client.find_children(parent_list=['钢铁', '电子'])
for k, v in children_dict.items():
    print(k)
    print(len(v))
    for i in v:
        print(i.id)

In [ ]:
query = {
    'parent': {
        '$in': ['钢铁', '电子']
    }
}
data = mongo_tags_client.tags_collection.find(query)
# for i in data:
#     print(i['id'])
# s = set()
# for k, v in itertools.groupby(data, key=lambda x: x['parent']):
#     print(k)
#     s.add(k)
# print(s)
dict((k, list(map(lambda x: Tag(id=x['id'], parent=x.get('parent', None), values = x.get('values', None)), values))) for k, values in itertools.groupby(sorted(data, key=lambda x: x['parent']), key=lambda x: x['parent']))